In [3]:
import requests
import json
import adal

tenant_id = "e066262f-e599-4384-9941-a1bd6e0068d1"
authority_url = 'https://login.microsoftonline.com/'+tenant_id+'/'
resource_url = 'powerbi://api.powerbi.com/v1.0/myorg/POC'     #XMLA WORKSAPCE


#client_secret = "2D7nW3v-XyCrUpMx-.aXo8cfP_pQ9Sjv5p"
user= ""
password=""
client_id = "9494ee7b-125c-49fb-b312-1a99158d1fb9"
context = adal.AuthenticationContext(authority=authority_url,
                                     validate_authority=True,
                                     api_version=None)
token = context.acquire_token_with_client_credentials(resource_url, client_id, user,password)

#acquire_token_with_username_password(resource, username, password, client_id)[source]

#token = context.acquire_token_with_username_password(resource_url,user,password, client_id)
access_token = token.get('accessToken')
print(access_token)

TypeError: acquire_token_with_client_credentials() takes 4 positional arguments but 5 were given

In [35]:

import adal
import requests
import json

tenant_id = "e066262f-e599-4384-9941-a1bd6e0068d1"
authority_url = f'https://login.microsoftonline.com/{tenant_id}/'
resource_url =  'https://analysis.windows.net/powerbi/api'     
client_secret = ""


client_id = ""
context = adal.AuthenticationContext(authority=authority_url,
                                     validate_authority=True,
                                     api_version=None)
token = context.acquire_token_with_client_credentials(resource_url, client_id, client_secret)

access_token = token['accessToken'] #.get#('accessToken')

print (access_token)


AdalError: Get Token request returned http error: 400 and server response: {"error":"invalid_resource","error_description":"AADSTS500011: The resource principal named powerbi://api.powerbi.com/v1.0/myorg/POC was not found in the tenant named e066262f-e599-4384-9941-a1bd6e0068d1. This can happen if the application has not been installed by the administrator of the tenant or consented to by any user in the tenant. You might have sent your authentication request to the wrong tenant.\r\nTrace ID: d76ba352-0d5b-4dd0-b72a-2fcc075d9600\r\nCorrelation ID: 130e45b0-bfb7-4a19-aae3-56a8bbc39337\r\nTimestamp: 2021-07-15 11:21:06Z","error_codes":[500011],"timestamp":"2021-07-15 11:21:06Z","trace_id":"d76ba352-0d5b-4dd0-b72a-2fcc075d9600","correlation_id":"130e45b0-bfb7-4a19-aae3-56a8bbc39337","error_uri":"https://login.microsoftonline.com/error?code=500011"}

In [29]:


workspaceID = ""
## get the datasetid's in the workspace

dataset_url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceID}/datasets'
header = {'Authorization': f'Bearer {access_token}','Content-Type':'application/json'}
datasets =  json.loads(requests.get(url=dataset_url, headers=header).content)['value']


        


In [30]:
for i in range(0,len(datasets)):

    print(datasets[i]["name"])


downloadme
untitled (1)


In [31]:


#get datasetid based on the name
requested_dataset = "downloadme"

for item in datasets:
    if item['name']==requested_dataset:
         requested_dataset_id = item['id']
    else:
        pass
    
#trigger the refresh
refresh_url = 'https://api.powerbi.com/v1.0/myorg/groups/' + workspaceID + '/datasets/' + requested_dataset_id + '/refreshes?effectiveDate=07%2F13%2F2021'
header = {'Authorization': f'Bearer {access_token}'}
r = requests.post(url=refresh_url, headers=header)

print(r.status_code)

if r.status_code==202:
    print(f'request to refresh dataset ""{requested_dataset}"" accepted')




202
request to refresh dataset ""downloadme"" accepted


In [32]:
import time
#import pandas as pd

refresh_status_url= f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceID}/datasets/{requested_dataset_id}/refreshes?$top=1'  
header = {'Authorization': f'Bearer {access_token}'}
r_status = json.loads(requests.get(url=refresh_status_url, headers=header).content)



refresh_status=  r_status['value'][0]['status']
while refresh_status == 'Unknown' :
        print(f'status is {refresh_status}, retrying in 60 seconds')
        time.sleep(60)   #sleep 60 seconds
        r_status = json.loads(requests.get(url=refresh_status_url, headers=header).content)
        refresh_status=  r_status['value'][0]['status']

if refresh_status   == 'Completed' :
    print ('Refresh Succesful')
elif   refresh_status   == 'Failed' :  
     print ('Refresh Failed')
      
    

df= pd.json_normalize(r_status['value']).head(10)
df

status is Unknown, retrying in 60 seconds
status is Unknown, retrying in 60 seconds
status is Unknown, retrying in 60 seconds
status is Unknown, retrying in 60 seconds
status is Unknown, retrying in 60 seconds
Refresh Succesful


,id,refreshType,startTime,endTime,status,requestId
0,816129636,ViaApi,2021-07-15T11:04:12.767Z,2021-07-15T11:09:30.097Z,Completed,f63a2023-f85c-41fe-8e0b-d174f0a18af2


In [33]:
refresh_status_url= f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceID}/datasets/{requested_dataset_id}/refreshes?$top=1'  
header = {'Authorization': f'Bearer {access_token}'}
r_status = requests.get(url=refresh_url, headers=header)

print(r_status.content)



b'{\r\n  "@odata.context":"http://wabi-us-north-central-b-redirect.analysis.windows.net/v1.0/myorg/groups/9fdfba01-6d6e-41e4-80e6-589c47ba033d/$metadata#refreshes","value":[\r\n    {\r\n      "id":816129636,"refreshType":"ViaApi","startTime":"2021-07-15T11:04:12.767Z","endTime":"2021-07-15T11:09:30.097Z","status":"Completed","requestId":"f63a2023-f85c-41fe-8e0b-d174f0a18af2"\r\n    },{\r\n      "id":816117714,"refreshType":"ViaApi","startTime":"2021-07-15T10:53:17.567Z","endTime":"2021-07-15T10:55:19.52Z","status":"Completed","requestId":"835ca01c-e899-498e-b654-ea19b882ef6d"\r\n    },{\r\n      "id":816115761,"refreshType":"ViaApi","startTime":"2021-07-15T10:51:01.95Z","endTime":"2021-07-15T10:53:01.513Z","status":"Completed","requestId":"4fc65e17-056b-4f80-a8bb-581157dec406"\r\n    },{\r\n      "id":816114940,"refreshType":"ViaApi","startTime":"2021-07-15T10:49:02.907Z","endTime":"2021-07-15T10:51:01.84Z","status":"Completed","requestId":"9f2b5769-e010-4c22-8df2-ccc30f5aa82a"\r\n    

In [158]:
import smtplib, ssl

port = 587  # For starttls
smtp_server = "smtp.office365.com"
sender_email = ""
receiver_email = ""
password = "" #input("Type your password and press enter:")
message = """\
Subject: Look! Im using python!

This message is sent from Python

""" 


context = ssl.create_default_context()
with smtplib.SMTP(smtp_server, port) as server:
    server.starttls(context=context)
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)
    server.quit()


In [159]:
import pyodbc
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']